<a href="https://colab.research.google.com/github/ShaifaliKhulbe/Project_1/blob/main/final_tweets%2Bretweets%2Bquote_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import the libraries.
import os
import tweepy as tw
import pandas as pd
import numpy as np
import configparser
import json
from tqdm import tqdm

config = configparser.ConfigParser()
config.read('Config.ini')


# Assigning Twitter Developer API’s and Access Tokens to variables.
api_key = config['twitter']['api_key']
api_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret'] 


# Authenticate the API’s and Access Tokens.
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Define the search words and start date as variables. 
#search_words = "football"
date_since = "2020-01-01"

In [2]:
def quote_tweets(name, tweet_id, tweet_url):


  replies=[] #replies to original tweet
  quotes=[] #tweets with true quote status
  matching_quotes = [] #tweet
  quote_ids = [] #tweet id of tweet being quoted
  objects = [] #tweet metadata
  all_tweets=[] #all tweets returned
  for tweet in tw.Cursor(api.search,q=tweet_url, result_type='recent', timeout=999999).items(3000):
    #Checks if tweet is a tweet reply to the specific tweet
      all_tweets.append(tweet)
      if hasattr(tweet, 'in_reply_to_status_id_str'):
          if (tweet.in_reply_to_status_id_str==tweet_id):
              replies.append(tweet)
      #if hasattr(tweet, 'quoted_status'):
      if tweet.is_quote_status == True:
        quotes.append(tweet) #Full Tweet objects with all the meta data
        quote_ids.append(tweet.quoted_status_id) #These should all be the exact same
        if str(tweet_id) == str(tweet.quoted_status_id):
          objects.append(tweet)
          matching_quotes.append(tweet.text)
      

  return objects

Resources: 

https://colab.research.google.com/drive/1jLOCy0LdK396lTth44uTqNmV3WbaAQ4f?usp=sharing#scrollTo=2eyHQIWn2t1w

In [6]:
def get_related_tweets(key_word, date_since):


    tweet_user = []
    tweet_time = []
    tweet_string = []
    tweet_location = [] 
    tweet_followers = []
    tweet_following = []
    tweet_like = []
    tweet_hashtags = []
    tweet_description = []
    tweet_totaltweets = []
    tweet_retweet_count = []
    tweet_id = []

    retweet_user = []
    retweet_time = []
    retweet_location = [] 
    retweet_followers = []
    retweet_following = []
    retweet_hashtags = []
    retweet_description = []
    retweet_string = []
    retweet_id = []
    
    quote_tweet_user = []
    quote_tweet_time = []
    quote_tweet_location = [] 
    quote_tweet_followers = []
    quote_tweet_following = []
    quote_tweet_hashtags = []
    quote_tweet_description = []
    quote_tweet_string = []
    quote_tweet_id = []


    for tweet in tqdm(tw.Cursor(api.search, lang = "en", q=key_word, since = date_since, count=10).items(10)):
            if (not tweet.retweeted) and ('RT @' not in tweet.text):
                if tweet.lang == "en":

                    #COLLECTING INFO ON TWEETS

                    tweet_user.append(tweet.user.name)
                    tweet_time.append(tweet.created_at)
                    tweet_string.append(tweet.text)
                    tweet_location.append(tweet.user.location)
                    tweet_followers.append(tweet.user.followers_count)
                    tweet_following.append(tweet.user.friends_count)
                    tweet_like.append(tweet.favorite_count)           
                    tweet_id.append(tweet.id_str)
                    tweet_description.append(tweet.user.description)
                    tweet_totaltweets.append(tweet.user.statuses_count)
                    tweet_retweet_count.append(tweet.retweet_count)

                    #extract hashtags
                    hashtags = tweet.entities['hashtags']
                    hashtext = list()
                    for i in range(0, len(hashtags)):
                      hashtext.append(hashtags[i]['text'])
                    tweet_hashtags.append(hashtext)


                    #COLLECTING INFO ON RETWEETS

                    retweeters = api.retweets(tweet.id_str)

                    for retweet in retweeters:                              
                      
                      #original tweet's id which has been retweeted
                      retweet_id.append(tweet.id_str)
                      retweet_string.append(retweet.text)              
                      retweet_user.append(retweet.user.name)
                      retweet_time.append(retweet.created_at)
                      retweet_description.append(retweet.user.description)
                      retweet_location.append(retweet.user.location)
                      retweet_followers.append(retweet.user.followers_count)
                      retweet_following.append(retweet.user.friends_count)
                      
                      #extract hashtags
                      hashtags = retweet.entities['hashtags']
                      hashtext = list()
                      for i in range(0, len(hashtags)):
                        hashtext.append(hashtags[i]['text'])
                      retweet_hashtags.append(hashtext)


                    #COLLECTING INFO ON QUOTE TWEETS

                    tweet_url = f"https://twitter.com/{tweet.user.screen_name}/status/{tweet.id}"


                    objects = quote_tweets(tweet.user.name , tweet.id_str, tweet_url)

                    print("Number of quote tweets: ", len(objects))

                    for qtweet in objects:
                        
                      print("TWEET.ID: " , tweet.id_str)
                      print("TWEET.TEXT: ", tweet.text)
                      print(qtweet.text)
                      print(qtweet.created_at)

    
                      quote_tweet_id.append(tweet.id_str)
                      quote_tweet_string.append(qtweet.text)              
                      quote_tweet_user.append(qtweet.user.name)
                      quote_tweet_time.append(qtweet.created_at)
                      quote_tweet_description.append(qtweet.user.description)
                      quote_tweet_location.append(qtweet.user.location)
                      quote_tweet_followers.append(qtweet.user.followers_count)
                      quote_tweet_following.append(qtweet.user.friends_count)
                      
                      #extract hashtags
                      hashtags = qtweet.entities['hashtags']
                      hashtext = list()
                      for i in range(0, len(hashtags)):
                        hashtext.append(hashtags[i]['text'])
                      quote_tweet_hashtags.append(hashtext)

  
    #DATAFRAME TO STORE TWEET INFORMATION            
    
    df_tweets = pd.DataFrame({'tweet_id': tweet_id, 'user_name':tweet_user, 'user_description': tweet_description, "total_user_tweets": tweet_totaltweets, 'time': tweet_time, 'tweet': tweet_string, 'location' : tweet_location, 'followers': tweet_followers,
          'following': tweet_following,'hashtags': tweet_hashtags,'likes': tweet_like, 'number_of_retweets': tweet_retweet_count })
    df_tweets.to_csv(f"{key_word}.csv")

    #DATAFRAME TO STORE RETWEET INFORMATION PER TWEET

    df_retweets = pd.DataFrame({'retweet_id' : retweet_id, 'retweet_text' : retweet_string, 'name':retweet_user, 'time': retweet_time,'location' : retweet_location, 'followers': retweet_followers,
                        'following': retweet_following,'hashtags': retweet_hashtags })    
    df_retweets.to_csv(f"{key_word}_retweets.csv") 

    #DATAFRAME TO STORE QUOTE_TWEET INFORMATION PER TWEET 
  
    df_qtweets = pd.DataFrame({'quote_tweet_id' : quote_tweet_id, 'quote_tweet_text' : quote_tweet_string, 'name':quote_tweet_user, 'time': quote_tweet_time,'location' : quote_tweet_location, 'followers': quote_tweet_followers,
                  'following': quote_tweet_following,'hashtags': quote_tweet_hashtags })    
    df_qtweets.to_csv(f"{key_word}_quote_tweets.csv")

    
    return df_tweets, df_retweets, df_qtweets




In [4]:
keywords_list = []

file_keywords = pd.read_csv (r'/content/keywords.csv')
for i in range(0, len(file_keywords)):
  keyword = file_keywords.iloc[i, 1]
  keywords_list.append(keyword.lower())


In [7]:
for keyword in tqdm(keywords_list): 
  # Call the function to get related tweets in csv files
  df_tweets, df_retweets, df_qtweets = get_related_tweets(keyword, date_since)

  0%|          | 0/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.88it/s]
2it [00:01,  1.43it/s]

Number of quote tweets:  0



3it [00:02,  1.35it/s]

Number of quote tweets:  0



4it [00:02,  1.52it/s]

...


6it [00:02,  3.02it/s]

Number of quote tweets:  0



10it [14:15, 85.53s/it]
 99%|█████████▉| 99/100 [1:30:33<04:19, 259.45s/it]

Number of quote tweets:  0



0it [00:00, ?it/s]
1it [00:00,  1.76it/s]
10it [00:01,  8.69it/s]
100%|██████████| 100/100 [1:30:34<00:00, 54.34s/it] 


In [5]:
print(keywords_list)

['culture', 'ai', 'artificialintelligence', 'machinelearning', 'deeplearning', 'art', 'arts', 'music', 'cryptocurrency', 'olympics', 'funny', 'crowdfunding', 'influencermarketing', 'opioid', 'healthinsurance', 'photography', 'vegan', 'ethereum', 'bitcoin', 'gold', 'movies', 'running', 'cryptocurrencies', 'java', 'python', 'blockchain', 'datascience', 'fintech', 'science', 'fitnessfacebook', 'youtube', 'amazon', 'weather', 'google', 'gmail', 'instagram', 'translate', 'ipl', 'wordle', 'bbcnews', 'flipkart', 'startbucks', 'cricket', 'fifaworldcup', 'qatar2022', 'tiktok', 'argentina', 'messi', 'worldcup', 'givingtuesday', 'christmas', 'fifaworldcup', 'elections', 'bollywood', 'beinghuman', 'elonmusk', 'dogecoin', 'pokemon', 'nintendoswitch', 'spirituality', 'soul', 'aliens', 'earth', 'water', 'flood', 'naturaldisaster', 'religion', 'god', 'satan', 'entrepreneur', 'hinduism', 'islam', 'christinity', 'jew', 'buddhism', 'home', 'house', 'realestate', 'truth', 'energy', 'racism', 'gratitude', 